In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
import theano
import theano.tensor as T
import lasagne
from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.objectives import categorical_crossentropy, aggregate
import sys
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

/home/rishikesh/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/rishikesh/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN 5105)
/home/rishikesh/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you s

In [4]:
coupons_train=pd.read_csv("Translated_Data/coupon_list_train.csv", parse_dates=["DISPFROM","DISPEND"])
coupons_test = pd.read_csv("Translated_Data/coupon_list_test.csv")

In [5]:
coupons_train["DISPFROM"].fillna(pd.Timestamp("19000101"), inplace=True)
coupons_train = coupons_train.sort(columns=["DISPFROM"]).reset_index(drop=True)

/home/rishikesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [ ]:
coupons_visit = pd.read_csv("Translated_Data/coupon_visit_train.csv", parse_dates=["I_DATE"])
coupons_visit = coupons_visit.sort(columns=["I_DATE"]).reset_index(drop=True)

In [ ]:
coupons_visit = coupons_visit.rename(columns={'VIEW_COUPON_ID_hash': 'COUPON_ID_hash'})

In [6]:
def gen_validation(coupons_train,time_delta):
    max_date = coupons_train["DISPFROM"].max()
    valid_start = max_date - time_delta
    coupons_valid = coupons_train[(coupons_train["DISPFROM"] > valid_start)]
    coupons_train = coupons_train[~ (coupons_train["DISPFROM"] > valid_start)]
    return coupons_train,coupons_valid

In [7]:
coupons_train,coupons_valid=gen_validation(coupons_train,datetime.timedelta(days=0))

In [8]:
def remove_outliers(coupons_train,coupons_valid):
    if len(coupons_valid)>0:
        very_long_time_display=coupons_valid[coupons_valid.DISPPERIOD > 20].COUPON_ID_hash
        very_low_price = coupons_valid[coupons_valid.DISCOUNT_PRICE <= 100].COUPON_ID_hash
        coupons_valid = coupons_valid[~coupons_valid.COUPON_ID_hash.isin(very_long_time_display)]
        coupons_valid = coupons_valid[~coupons_valid.COUPON_ID_hash.isin(very_low_price)].reset_index(drop=True)
        
    very_long_time_display = coupons_train[coupons_train.DISPPERIOD > 20].COUPON_ID_hash
    coupons_train = coupons_train[~coupons_train.COUPON_ID_hash.isin(very_long_time_display)].reset_index(drop=True)
    
    return coupons_train,coupons_valid

In [9]:
coupons_train,coupons_valid=remove_outliers(coupons_train,coupons_valid)

In [ ]:
coupons_train["PREF_NAME"].unique()

In [ ]:
user_frame["PREF_NAME"].unique()

In [10]:
 def preprocess(df):
        df["REDUCE_PRICE"] = df["CATALOG_PRICE"] - df["DISCOUNT_PRICE"]
        for key in ["DISCOUNT_PRICE", "CATALOG_PRICE", "REDUCE_PRICE"]:
            df[key + "_LOG"] = np.log(df[key] + 1.0).astype(np.float32)

        df["VALIDPERIOD_NA"] = np.array(pd.isnull(df["VALIDPERIOD"]), dtype=np.int32)
        df["DISPPERIOD_C"] = np.array(df["DISPPERIOD"].clip(0, 8), dtype=np.int32)
        df["PRICE_RATE"] = np.array(df.PRICE_RATE, dtype=np.float32)
        df["large_area_name"].fillna("NA", inplace=True)
        df["ken_name"].fillna("NA", inplace=True)
        df["small_area_name"].fillna("NA", inplace=True)
        df["LARGE_AREA_NAME"] = df["large_area_name"]
        df["PREF_NAME"] = df["ken_name"]
        df["SMALL_AREA_NAME"] = df["large_area_name"] + ":" + df["ken_name"] + ":" + df["small_area_name"]
        df["CATEGORY_NAME"] = df["CAPSULE_TEXT"] + df["GENRE_NAME"]

        usable_dates = ['USABLE_DATE_MON',
                        'USABLE_DATE_TUE',
                        'USABLE_DATE_WED',
                        'USABLE_DATE_THU',
                        'USABLE_DATE_FRI',
                        'USABLE_DATE_SAT',
                        'USABLE_DATE_SUN',
                        'USABLE_DATE_HOLIDAY',
                        'USABLE_DATE_BEFORE_HOLIDAY']        
        for key in usable_dates:
            df[key].fillna(0, inplace=True)
        
        df["USABLE_DATE_SUM"] = 0
        for key in usable_dates:
            df["USABLE_DATE_SUM"] += df[key]
        
        cols = df.columns.tolist()
        cols.remove("DISPFROM")
        cols.remove("DISPEND")
        for key in cols:
            df[key].fillna("NA", inplace=True)
        return df

In [11]:
coupons_train=preprocess(coupons_train)
coupons_valid=preprocess(coupons_valid)
coupons_test=preprocess(coupons_test)

In [47]:
 coupon_mapper = DataFrameMapper([
                ('CATEGORY_NAME', LabelBinarizer()),
                ('PRICE_RATE', None),
                ('CATALOG_PRICE_LOG', None),
                ('DISCOUNT_PRICE_LOG', None),
                ('REDUCE_PRICE_LOG', None),
                ('DISPPERIOD_C', LabelBinarizer()),
                ('VALIDPERIOD_NA', LabelBinarizer()),
                ('USABLE_DATE_SUM', None),
                ('LARGE_AREA_NAME', LabelBinarizer()),
                ('SMALL_AREA_NAME', LabelBinarizer()),
                ('PREF_NAME', LabelBinarizer()),
                ])

In [48]:
coupon_mapper.fit(pd.concat([coupons_train, coupons_valid, coupons_test]))
train_coupon_vec = coupon_mapper.transform(coupons_train.copy())
#valid_coupon_vec = coupon_mapper.transform(coupons_valid.copy())
test_coupon_vec = coupon_mapper.transform(coupons_test.copy())

len PREF_NAME = 47 

In [14]:
user_frame = pd.read_csv("Translated_Data/user_list.csv")
details_frame = pd.read_csv("Translated_Data/coupon_detail_train.csv",parse_dates=["I_DATE"])
details_frame = details_frame.sort(columns=["I_DATE"]).reset_index(drop=True)

/home/rishikesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [15]:
user_mapper = DataFrameMapper([
                ('SEX_ID', LabelBinarizer()),
                ('PREF_NAME', LabelBinarizer()),
                ('AGE', None),
                ])

In [16]:
user_frame["PREF_NAME"].fillna("NA", inplace=True)
user_vec = user_mapper.fit_transform(user_frame.copy())

In [ ]:
user_frame['PREF_NAME'][0]

In [ ]:
for i in range(len(user_vec[0][:])):
    if user_vec[0][i] == 1:
        print i

In [ ]:
users = []
coupons_train["ROW_ID"] = pd.Series(coupons_train.index.tolist())
#coupons_valid["ROW_ID"] = pd.Series(coupons_valid.index.tolist())

for i, user in user_frame.iterrows():
    if i % 100 == 0:
        sys.stdout.write("\r{0}".format(i))
        sys.stdout.flush() 
    coupons = details_frame[details_frame.USER_ID_hash.isin([user["USER_ID_hash"]])]
    train_coupon_data = pd.merge(coupons[["COUPON_ID_hash","ITEM_COUNT","I_DATE"]],
                                         coupons_train,
                                         on="COUPON_ID_hash", how='inner',
                                         suffixes=["_x",""], copy=False)
    train_coupon_data = train_coupon_data.sort(columns=["I_DATE"])
    row_ids = train_coupon_data.ROW_ID.unique().tolist()
    `
    #coupons_visited = coupons_visit[coupons_visit.USER_ID_hash.isin([user["USER_ID_hash"]])]
    #train_coupon_visited_data = pd.merge(coupons_visited[["COUPON_ID_hash","PURCHASE_FLG"]],
                                         #coupons_train,
                                         #on="COUPON_ID_hash", how='inner',
                                         #suffixes=["_x",""], copy=False)
    #train_coupon_visited_data = train_coupon_visited_data.sort(columns=["I_DATE"])
    #tmp = train_coupon_visited_data[train_coupon_visited_data.PURCHASE_FLG>0].ROW_ID.unique()
    #visited_row_ids = [i for i in train_coupon_visited_data.ROW_ID.unique() if i not in tmp]
    

    #valid_coupon_data = pd.merge(coupons[["COUPON_ID_hash","ITEM_COUNT","I_DATE"]],
                                         #coupons_valid, on="COUPON_ID_hash",
                                         #how='inner', suffixes=["_x",""], copy=False)
    #valid_coupon_data = valid_coupon_data.sort(columns=["I_DATE"])
    #valid_row_ids = valid_coupon_data.ROW_ID.unique().tolist()

    users.append({"user": user_vec[i],
                    "coupon_ids": row_ids})
                    #"valid_coupon_ids": valid_row_ids,
                    #"visited_row_ids": visited_row_ids})

In [17]:
import cPickle

In [21]:
## Used to load user so that don't have to generate again
f = open('user_all.obj','rb')
users = cPickle.load(f)
f.close()

In [23]:
users[0]

{'coupon_ids': [15101],
 'user': array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 25])}

For Bucketizing the age.
Does not work. 

Dont run the next 2 blocks

In [51]:
NUM_BUCKET = 6
def bucket(age):
    r = np.zeros(NUM_BUCKET)
    if age < 30:
        r[0] = 1
    elif age > 50:
        r[-1] = 1
    else:
        r[(age/5)-5] = 1
    #r[age/10-1] = 1
    return r

In [60]:
users[1]['user'].shape

(55,)

In [56]:
for i in xrange(len(users)):
    users[i]['user'] = np.append(users[i]['user'][:-1],bucket(users[i]['user'][-1]))

In [ ]:
## Storing user array for further use
f = open('user_mod_all.obj', 'wb')
cPickle.dump(users, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [61]:
def maxmin_columns(coupons_train,coupon_ids):
    return coupons_train.ix[
        coupon_ids, ("CATALOG_PRICE","DISCOUNT_PRICE")
        ].as_matrix().astype(np.float32)

Added mean of unpurchased coupon

Number of unpurchased coupons

Ratio of purchased to unpurchased

In [62]:
def purchase_history_features(train_coupon_vec, user_coupon_vec, user_coupon_not_purchased_vec, maxmin_columns, 
                              filter_idx=None, n_filter_idx=None):
        sum_vec = np.zeros(2, dtype=np.float32)
        maxmin_vec = np.zeros((4), dtype=np.float32)
        mean_coupon_vec = np.zeros(len(train_coupon_vec[0]), dtype=np.float32)
        #mean_coupon_not_purchased_vec = np.zeros(len(train_coupon_vec[0]), dtype=np.float32)
        
        #lennp = 0
        
        if filter_idx is not None:
            '''
            if len(user_coupon_not_purchased_vec[n_filter_idx]) > 0:
                lennp = len(user_coupon_not_purchased_vec[n_filter_idx])
                mean_coupon_not_purchased_vec[:] = user_coupon_not_purchased_vec[n_filter_idx].mean(0)
            '''
            if len(user_coupon_vec[filter_idx]) > 0:
                mean_coupon_vec[:] = user_coupon_vec[filter_idx].mean(0)
                sum_vec[0] = filter_idx.sum()
                sum_vec[1] = np.log(sum_vec[0] + 1.0)
                max_val = maxmin_columns[filter_idx].max(0)
                min_val = maxmin_columns[filter_idx].min(0)
                maxmin_vec[0] = max_val[0]
                maxmin_vec[1] = min_val[0]
                maxmin_vec[2] = max_val[1]
                maxmin_vec[3] = min_val[1]
        else:
            '''
            if len(user_coupon_not_purchased_vec) > 0:
                lennp = len(user_coupon_not_purchased_vec)
                mean_coupon_not_purchased_vec[:] = user_coupon_not_purchased_vec.mean(0)
            '''
            if len(user_coupon_vec) > 0:
                mean_coupon_vec = user_coupon_vec.mean(0)
                sum_vec[0] = len(user_coupon_vec)
                sum_vec[1] = np.log(sum_vec[0] + 1.0)
                max_val = maxmin_columns.max(0)
                min_val = maxmin_columns.min(0)
                maxmin_vec[0] = max_val[0]
                maxmin_vec[1] = min_val[0]
                maxmin_vec[2] = max_val[1]
                maxmin_vec[3] = min_val[1]
        '''        
        ratio = 0
        if lennp != 0:
            ratio = sum_vec[0]*1.0/lennp
        '''
        #return np.hstack((mean_coupon_vec, mean_coupon_not_purchased_vec, sum_vec, maxmin_vec, 
                          #lennp, ratio))
        return np.hstack((sum_vec, maxmin_vec)),mean_coupon_vec

In [63]:
COUPON_DISP_NEAR = 400
COUPON_DISP_NEAR_MIN = 10

In [66]:
def gen_train_data(num_nega=2, verbose=True):
        x = []
        y = []
        count = 0
        for user in users:
            if count % 100 == 0:
                sys.stdout.write("\r{0}/{1}".format(count, len(users)))
                sys.stdout.flush()
            count += 1
            coupon_ids = np.array(user["coupon_ids"], dtype=np.int32)
            user_coupons = train_coupon_vec[coupon_ids]
            
            '''
            coupon_ids_np = np.array(user["visited_row_ids"], dtype=np.int32)
            user_coupon_np = train_coupon_vec[coupon_ids_np]
            '''
            maxmin_c = maxmin_columns(coupons_train,coupon_ids)
            for i in xrange(len(user_coupons)):
                target_coupon_vec = user_coupons[i]
                rid = coupon_ids[i]
                nega_list = range(max(0, rid - COUPON_DISP_NEAR), rid)
                if len(nega_list) < COUPON_DISP_NEAR_MIN:
                    continue

                filter_idx = np.ones(user_coupons.shape[0], dtype=np.bool)
                
                # exclude coupons that was purchased after the target coupon
                filter_idx[i:] = False
                # exclude the target coupon (and remove duplicate)
                filter_idx[coupon_ids == coupon_ids[i]] = False
               
                '''
                filter_idx_np = np.ones(user_coupon_np.shape[0], dtype=np.bool)
                for ix in range(len(filter_idx_np)):
                    if user_coupon_np[ix] in user_coupons:
                        filter_idx_np[ix] = False
                
                index = np.where(coupon_ids_np == coupon_ids[i])[0]
                if len(index) > 0:
                    filter_idx_np[index[0]:] = False
                filter_idx_np[coupon_ids_np in coupon_ids] = False
                '''
                hist_feat, mean = purchase_history_features(train_coupon_vec,user_coupons, user_coupons,
                                                             maxmin_c,
                                                             filter_idx, filter_idx)
                
                # dot product(both elementwise and overall) for similarity 
                # between mean purchased coupons and current coupon
                
                sim_coupon_vec = mean*target_coupon_vec
                sim_score = np.dot(mean, target_coupon_vec)
                
                #cos_sim = cosine_similarity(mean.reshape((-1,1)).T, target_coupon_vec.reshape((-1,1)).T)[0]
                #euc_sim = euclidean_distances(mean.reshape((-1,1)).T, target_coupon_vec.reshape((-1,1)).T)[0]
                
                area_sim = np.dot(target_coupon_vec[-47:], np.delete(user["user"][1:-6], 26))
                #sim_coupon_np_vec = hist_feat[150:300]*target_coupon_vec
                
                # feature vector (user_feature + purchase_history_feature + coupon_feature)
                purchased_feat = np.hstack((user["user"], hist_feat, target_coupon_vec, 
                                            sim_coupon_vec, sim_score, area_sim))
                                            #, sim_coupon_np_vec))
                x.append(purchased_feat)
                y.append([1]) # posi

                # select random unpurchased coupons
                for j in xrange(num_nega):
                    found = False
                    for _ in xrange(10):
                        unpurchased_idx = np.random.choice(nega_list, 1)[0]
                        if unpurchased_idx not in user["coupon_ids"]:
                            found = True
                            break
                    if found:
                        
                        sim_coupon_vec = mean*train_coupon_vec[unpurchased_idx]
                        sim_score = np.dot(mean, train_coupon_vec[unpurchased_idx])
                        
                        #cos_sim = cosine_similarity(mean.reshape((-1,1)).T, 
                        #                            train_coupon_vec[unpurchased_idx].reshape((-1,1)).T)[0]
                        #euc_sim = euclidean_distances(mean.reshape((-1,1)).T,
                        #                            train_coupon_vec[unpurchased_idx].reshape((-1,1)).T)[0]
                
                        area_sim = np.dot(train_coupon_vec[unpurchased_idx][-47:], 
                                          np.delete(user["user"][1:-6], 26))
                        #sim_coupon_np_vec = hist_feat[150:300]*train_coupon_vec[unpurchased_idx]
                        
                        unpurchased_feat = np.hstack((user["user"],
                                                      hist_feat,
                                                      train_coupon_vec[unpurchased_idx], 
                                                      sim_coupon_vec, sim_score,
                                                      #euc_sim, cos_sim, 
                                                      area_sim))#, sim_coupon_np_vec))
                        x.append(unpurchased_feat)
                        y.append([0]) # nega
                        
        x = np.array(x, dtype=np.float32)
        y = np.array(y, dtype=np.int32)

        return x, y

In [28]:
def gen_valid_data(num_nega=5, verbose=True):
        x = []
        y = []
        count = 0
        for user in users:
            if count % 100 == 0:
                sys.stdout.write("\r{0}/{1}".format(count, len(users)))
                sys.stdout.flush()
            count += 1
            coupon_ids = np.array(user["coupon_ids"], dtype=np.int32)
            user_coupons = train_coupon_vec[coupon_ids]
            
            '''
            coupon_ids_np = np.array(user["visited_row_ids"], dtype=np.int32)
            user_coupon_np = train_coupon_vec[np.setdiff1d(coupon_ids_np,coupon_ids)]
            '''
            
            valid_coupon_ids=np.array(user["valid_coupon_ids"], dtype=np.int32)
            valid_user_coupons=valid_coupon_vec[valid_coupon_ids]
            maxmin_c = maxmin_columns(coupons_train,coupon_ids)
            
            hist_feat = purchase_history_features(train_coupon_vec, user_coupons, user_coupons, maxmin_c)
            
            for i in xrange(len(valid_user_coupons)):
                target_coupon_vec = valid_user_coupons[i]
                #rid = coupon_ids[i]
                sim_coupon_vec = hist_feat[:150]*target_coupon_vec
                sim_coupon_np_vec = hist_feat[150:300]*target_coupon_vec
                #sim_score = np.dot(hist_feat[:150], target_coupon_vec)
                
                # feature vector (user_feature + purchase_history_feature + coupon_feature)
                purchased_feat = np.hstack((user["user"], hist_feat, target_coupon_vec,
                                            sim_coupon_vec, sim_score))#, sim_coupon_np_vec))
                x.append(purchased_feat)
                y.append([1]) # posi

                # select random unpurchased coupons
                for j in xrange(num_nega):
                    found = False
                    for _ in xrange(10):
                        unpurchased_idx = np.random.choice(range(len(valid_coupon_vec)), 1)[0]
                        if unpurchased_idx not in user["valid_coupon_ids"]:
                            found = True
                            break
                    if found:
                        sim_coupon_vec = hist_feat[:150]*valid_coupon_vec[unpurchased_idx]
                        sim_score = np.dot(hist_feat[:150], valid_coupon_vec[unpurchased_idx])
                        #sim_coupon_np_vec = hist_feat[150:300]*valid_coupon_vec[unpurchased_idx]
                        unpurchased_feat = np.hstack((user["user"],
                                                      hist_feat,
                                                      valid_coupon_vec[unpurchased_idx], 
                                                      sim_coupon_vec, sim_score))#, sim_coupon_np_vec))
                        x.append(unpurchased_feat)
                        y.append([0]) # nega

        x = np.array(x, dtype=np.float32)
        y = np.array(y, dtype=np.int32)

        return x, y

In [ ]:
train_feature,train_label=gen_train_data()

17200/22873

In [ ]:
valid_feature,valid_label=gen_valid_data()


f = open('train_label.obj', 'rb')
train_label = cPickle.load(f)
f.close()

f = open('valid_label.obj', 'rb')
valid_label = cPickle.load(f)
f.close()

In [21]:
f = open('train__mod_feat.obj', 'rb')
train_feature = cPickle.load(f)
f.close()
f = open('train_mod_label.obj', 'rb')
train_label = cPickle.load(f)
f.close()

In [ ]:
f = open('valid_feat.obj', 'rb')
valid_feature = cPickle.load(f)
f.close()
f = open('valid_label.obj', 'rb')
valid_label = cPickle.load(f)
f.close()

In [25]:
f = open('train__mod_feat.obj', 'wb')
cPickle.dump(train_feature, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()
f = open('train_mod_label.obj', 'wb')
cPickle.dump(train_label, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [ ]:
f = open('valid_feat.obj', 'wb')
cPickle.dump(valid_feature, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()
f = open('valid_label.obj', 'wb')
cPickle.dump(valid_label, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [68]:
print train_feature.shape
#print valid_label.shape

(459231, 363)


In [ ]:
train_label = train_label.reshape((-1))
valid_label = valid_label.reshape((-1))

In [69]:
scaler = StandardScaler()
#scaler.fit(train_feature)

In [70]:

train_feature = scaler.fit_transform(train_feature)
#valid_feature = scaler.transform(valid_feature)

In [71]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

def build_mlp(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None, len(train_feature[0])),
                                     input_var=input_var)

    #l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.1)

    l_hid1 = lasagne.layers.DenseLayer(
            #l_in_drop, 
            l_in, 
            num_units=1300,
            nonlinearity=lasagne.nonlinearities.rectify)#,
            #W=lasagne.init.Normal(std=1.0))

    l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1_drop,
            #l_hid1, 
            num_units=80,
            nonlinearity=lasagne.nonlinearities.rectify)#,
            #W=lasagne.init.Normal())

    l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.1)

    l_out = lasagne.layers.DenseLayer(
            l_hid2_drop,
            #l_hid2, 
            num_units=1,
            nonlinearity=lasagne.nonlinearities.sigmoid)#,
            #W=lasagne.init.Normal())

    return l_out

In [72]:
input_var = T.matrix('inputs')
target_var = T.matrix('targets')

print("Building model and compiling functions...")
network = build_mlp(input_var)

prediction = lasagne.layers.get_output(network)

#l2_penalty = regularize_layer_params_weighted(layers, l2)
loss = lasagne.objectives.binary_crossentropy(prediction, target_var)
loss = loss.mean()

train_predict_fn = theano.function([input_var], prediction)

params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=1e-2, momentum=0.9)

#updates = lasagne.updates.momentum(loss, params, learning_rate=0.01)

train_fn = theano.function([input_var, target_var], loss, updates=updates)

test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.binary_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(test_prediction >= 0.5, target_var))

val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
test_predict_fn = theano.function([input_var], test_prediction >= 0.5)
test_predict_probs_fn = theano.function([input_var], test_prediction)

Building model and compiling functions...


In [ ]:
train_feature = []
train_label = []

In [73]:
# Finally, launch the training loop.
print("Starting training...")
sys.stdout.flush()
num_epochs = 5
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    '''
    if epoch % 5 == 0:
        del train_feature
        del train_label
        train_feature,train_label=gen_train_data()
        train_feature = scaler.transform(train_feature)
    '''
    for batch in iterate_minibatches(train_feature, train_label, 128, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        err, acc = val_fn(inputs, targets)
        train_acc += acc
        train_batches += 1
        #break

    # And a full pass over the validation data:
    '''
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(valid_feature, valid_label, 128, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1
        #break
    '''
    # Then we print the results for this epoch:
    print("\nEpoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches * 100))
    sys.stdout.flush()
    '''
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    '''
    '''
    if (epoch+1) % 5 == 0:
        np.savez('models_area/model_all,1300,80,'+str(epoch+1)+',bothSim,noUnvisited,noSampling,noMean,areasim.npz', 
                 *lasagne.layers.get_all_param_values(network))
        user_predictions={}
        start = time.time()
        predict_for_test()
        with open('models_area/output'+str(epoch+1)+'.csv', 'w') as f:
            f.write('USER_ID_hash,PURCHASED_COUPONS\n')
            for user in user_predictions.keys():
                f.write(user+','+' '.join(user_predictions[user])+'\n')
        print  
        print time.time() - start
    '''
    #    f = open('models/model_'+str(epoch)+'.obj', 'wb')
    #    cPickle.dump(network, f, protocol=-1)
    #    f.close()
    #    np.savez('models/model_'+str(epoch)+'.npz', *lasagne.layers.get_all_param_values(network))
    #break

Starting training...

Epoch 1 of 5 took 191.076s
  training loss:		0.472017
  training accuracy:		77.931375

Epoch 2 of 5 took 191.739s
  training loss:		0.444714
  training accuracy:		78.938484

Epoch 3 of 5 took 185.063s
  training loss:		0.435491
  training accuracy:		79.369294

Epoch 4 of 5 took 182.493s
  training loss:		0.429568
  training accuracy:		79.643722

Epoch 5 of 5 took 182.874s
  training loss:		0.425131
  training accuracy:		79.790084


In [ ]:
n_estimators = 2
D = [1/len(train_feature)]
for i in range(n_estimators):
    

In [ ]:
np.savez('models/model_all,1300,80,5,bothSim,noUnvisited,noSampling,NoMean.npz', 
         *lasagne.layers.get_all_param_values(network))

In [ ]:
f = open('models/model_100.obj', 'wb')
cPickle.dump(network, f, protocol=-1)
f.close()

In [ ]:
np.savez('models/model_25.npz', *lasagne.layers.get_all_param_values(network))

In [ ]:
lasagne.layers.get_all_param_values(network)[0][0]

In [ ]:
with np.load('models/model_20.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
lasagne.layers.set_all_param_values(network, param_values)

In [ ]:
vl = test_predict_fn(valid_feature)
err = 0
c=0
for i in xrange(len(vl)):
    if vl[i] == 1:
        c+=1
    if valid_label[i] != vl[i]:
        err += 1
print err*1.0/len(vl)

In [ ]:
print len(vl)

In [ ]:
for batch in iterate_minibatches(train_feature, train_label, 10, shuffle=True):
    inputs, targets = batch
    print test_predict_fn(inputs)
    print targets
    print test_predict_probs_fn(inputs)
    print val_fn(inputs, targets)
    break

In [ ]:
val_err

In [74]:
def predict_for_test() :
    test_user_coupons=test_coupon_vec
    user_id=0
    for user in users:
        coupon_ids = np.array(user["coupon_ids"], dtype=np.int32)
        user_coupons = train_coupon_vec[coupon_ids]
        
        '''
        coupon_ids_np = np.array(user["visited_row_ids"], dtype=np.int32)
        user_coupon_np = train_coupon_vec[np.setdiff1d(coupon_ids_np,coupon_ids)]
        '''
        
        maxmin_c = maxmin_columns(coupons_train,coupon_ids)
        hist_feat, mean = purchase_history_features(train_coupon_vec, user_coupons, user_coupons, maxmin_c)
        
        sim_coupon_vec = mean*test_coupon_vec
        sim_score = np.dot(mean,test_coupon_vec.T).reshape((-1,1))
        
        #cos_sim = cosine_similarity(mean.reshape((-1,1)).T, test_coupon_vec).T
        #euc_sim = euclidean_distances(mean.reshape((-1,1)).T, test_coupon_vec).T
        #print cos_sim.shape
        #print euc_sim.shape
        
        area_sim = np.dot(np.delete(user["user"][1:-6], 26), test_coupon_vec[:,-47:].T).reshape((-1,1))
        #print area_sim.shape
        #sim_coupon_np_vec = hist_feat[150:300]*test_coupon_vec
        
        user_feat=np.hstack((user["user"],hist_feat))
        user_feat_rep=np.array([list(user_feat)]*len(test_coupon_vec))
        
        purchased_feats=np.hstack((user_feat_rep,test_coupon_vec,sim_coupon_vec, sim_score,  
                                   #euc_sim, cos_sim, 
                                   area_sim))
        
        purchased_feats = scaler.transform(purchased_feats)
        ## TODO predict using purchased_feats
        conf_predicted = test_predict_probs_fn(purchased_feats)
        #print conf_predicted
        #print test_predict_fn(purchased_feats)
        #conf_predicted = train_predict_fn(purchased_feats)
        
        conf_predicted=[(conf_predicted[i],i) for i in range(len(conf_predicted))]
        conf_predicted.sort()
        top10=[ i for (c,i) in conf_predicted[-10:] ]
        top10.reverse()
        
        #print top10
        
        user_predictions[user_frame['USER_ID_hash'][user_id]]=[coupons_test['COUPON_ID_hash'][i] for i in top10]
        user_id+=1
        
        #if user_id == 2:
        #break
        
        if user_id%100==0:
            sys.stdout.write("\r{0}/{1}".format(user_id, len(users)))
            sys.stdout.flush()

In [75]:
user_predictions={}
start = time.time()
predict_for_test()
print  
print time.time() - start

22800/22873
907.143246889


In [ ]:
train_feature.shape

In [ ]:
print len(user_predictions[user_predictions.keys()[0]])
print len(user_predictions[user_predictions.keys()[1]])
print len(user_predictions[user_predictions.keys()[2]])
print len(user_predictions[user_predictions.keys()[3]])
print len(user_predictions[user_predictions.keys()[4]])

In [76]:
with open('output_area_age_bucket.csv', 'w') as f:
    f.write('USER_ID_hash,PURCHASED_COUPONS\n')
    for user in user_predictions.keys():
        f.write(user+','+' '.join(user_predictions[user])+'\n')